# Получение данных с форума Винского
> В этом ноутбуке я парсю сообщения из темы __[Статистика получения визы в Германию россиянами 2021/22
](https://forum.awd.ru/viewtopic.php?%3Ef=521&t=397637)__ форума Винского, где люди рассказывают о том, какую визу они получили с какой визовой историей и тд.
>Полученные данные я потом немного дорабатываю ~~напильником~~ руками и анализирую в ноутбуке __[germany_awd_visa_analysis.](https://github.com/MustDie-green/AWD-forum-visa-parsing/blob/main/germany_awd_visa_analysis.ipynb)__

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Функция для получения нужных страниц форума

In [2]:
def parse_forum(url, headers):
    result = []
    start = 0
    titles = []
    
    while True:
        request = requests.get(url = url + str(start), headers = headers)
        soup = BeautifulSoup(request.text, 'html.parser')
        title = soup.find('title')
        try:
            last_title = titles[-1]
        except:
            last_title = None
        
        if title == last_title:
            break
        else:
            result.append(soup)
            
        titles.append(title)
        start +=20
    return result

### Функция для спарсивания нужных кусков страницы  
`del` нужен, чтобы удалить закрепленный опрос в начале каждой страницы, который тоже спарсивается.

In [3]:
def parse_soups(soups, tag, class_, delete = True):
    result = []
    for el in soups:
        content = el.find_all(tag, class_ = class_)
        
        if delete:
            del content[0]
            
        for elem in content:
            result.append(elem.text.strip())
    return result

###  Парсим нужную тему форума 

In [4]:
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'cookie': '_ga_W0JDW2V29Y=GS1.1.1661181837.2.1.1661184512.60.0.0; _ga=GA1.2.1025230771.1661155454; _gid=GA1.2.721157209.1661155454; style_cookie=null; phpbb3_alft2_k=; phpbb3_alft2_sid=fa5f9f17fa55d5a1a82d62c62159899a; phpbb3_alft2_u=1'
}

soups = parse_forum('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=', headers)

### Парсим из страниц форума содержание и блок «автор», в котором содержится дата

In [5]:
contents = parse_soups(soups, 'div', 'content')
authors = parse_soups(soups, 'p', 'author')

Проверяем, все ли ок с длиной. Она должна соответствовать количеству сообщений на форуме.

In [6]:
print(len(contents))
print(len(authors))

172
172


### Функция для парсинга конкретных данных по регулярному выражению  

In [7]:
def get_data(regex, list, group_num = 2):
    result = []
    for value in list:
        reg = re.search(regex, value)
        if reg != None:
            result.append(reg.group(group_num).strip())
        elif reg == None:
            result.append('NaN')
        else:
            result.append('Произошло что-то странное')
    return result

### Парсим все нужные нам данные по регулярным выражениям  
  
Через `del` удаляем сообщение модератора, оно нам не нужно.  
В `number` генерируем номера сообщений на форуме, чтобы не потеряться. Не так элегантно, как спарсить номера со страниц, но гораздо проще.  
В регулярке есть важный момент — иногда люди пишут в середине каких-то строк дату типа «11.05.2022 получили визу», регулярки прописаны так, чтобы ловить куски от самой последней начальной цифры с точкой до самой первой конечной цифры с точкой. Так чаще всего оно ловит то, что нужно.

In [8]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
city = get_data('(1\.)(((?!1\.).)*?)(2\.)', contents)
visa_center = get_data('(2\.)(((?!2\.).)*?)(3\.)', contents) 
raw_history = get_data('(4\.)(((?!4\.).)*?)(5\.)', contents)
raw_limit = get_data('(5\.)(((?!5\.).)*?)(6\.)', contents)

del date[0]
del city[0]
del visa_center[0]
del raw_history[0]
del raw_limit[0]

number = list(range(2, len(date) + 2))


### Категоризуем данные визовой истории  
Хочу категоризовать на две группы:
- По длительности;
- По странам — Германия и не Германия.

#### По длительности

In [21]:
history = []

for text in raw_history:
    text = text.lower()
    if (
        'нет' in text
        ):
        history.append("Нет")
        
    elif (
        'полг' in text or
        'пол ' in text
        ):
        history.append('Полугодовые')
    
    elif (
        'лет' in text
        ):
        history.append('Годовые и больше')
        
    elif 'год' in text:
        history.append('Годовые и больше')
        
    elif (
        "мес" in text
        ):
        history.append('Меньше полугода')
        
    elif (
        'дней' in text or
        'кратко' in text or
        'дня' in text
        ):
        history.append('Меньше полугода')
        
    elif (
        'поездк' in text
        ):
        history.append('Даты поездки')
        
    elif (
        'чист' in text or
        'никогда' in text or
        'не бы' in text
        ):
        history.append('Нет')
        
    else:
        history.append('_Не пон_' + text)

### По странам
Тут я сделал достаточно тупо, оставил больше работы на ручную проверку.

In [10]:
history_country = []

for text in raw_history:
    text = text.lower()
    if (
        'немец' in text or
        'немц' in text or
        'герма' in text and
        'отка' not in text
        ):
        history_country.append("Германия в истории")
        
    elif (
        'чист' in text or
        'никогда' in text or
        'не бы' in text
        ):
        history_country.append('Нет')
        
    else:
        history_country.append('Другие страны')

### Категоризуем длину полученных виз

In [11]:
limit = []

for id, text in enumerate(raw_limit):
    text = text.lower()
    if (
        'отказ' in text
        ):
        limit.append("Отказ")
        
    elif (
        'лет' in text
        ):
        limit.append('Год и больше')
        
    elif (
        'пол' not in text and
        'год' in text
        ):
        limit.append('Год и больше')
        
    elif (
        'пол ' in text or
        'полг' in text
        ):
        limit.append('Полгода')
        
    elif (
        "мес" in text
        ):
        limit.append('Меньше полугода')
        
    elif (
        'дне' in text
        ):
        limit.append('Меньше полугода')
        
    elif (
        'поезд' in text
        ):
        limit.append('Даты поездки')
        
    else:
        limit.append(str(id) +'_Не пон_' + text)

### Собираем и сохраняем датафрейм

In [22]:
df = pd.DataFrame()

df['number'] = number
df['date'] = date
df['city'] = city
df['visa_center'] = visa_center
df['history'] = history
df['history_country'] = history_country
df['raw_history'] = raw_history
df['limit'] = limit
df['raw_limit'] = raw_limit

df.to_csv('raw_data.csv')

df

,number,date,city,visa_center,history,history_country,raw_history,limit,raw_limit
0,2,28 янв 2021,Москва,ВЦ Визаметрик,Полугодовые,Германия в истории,виза Германии на пол года,Год и больше,1 год
1,3,09 мар 2021,Екатеринбург,ВЦ Визаметрик,Годовые и больше,Германия в истории,Годовые чешская и две немецкие,Год и больше,3 года
2,4,26 мар 2021,Москва,ВЦ Визаметрик,Годовые и больше,Другие страны,"Испания, 1 год",Год и больше,3 года
3,5,19 май 2021,Москва,Визаметрик,Полугодовые,Германия в истории,"Чехия полгода, Германия год.",Год и больше,1 год
4,6,22 май 2021,Москва,Визаметрик,Нет,Нет,Не было виз,Меньше полугода,3 месяца
...,...,...,...,...,...,...,...,...,...
166,168,NaN,Москва,ВЦ Visametric на Киевской,Годовые и больше,Другие страны,Италия в 2020 году до пандемии однократная на ...,Меньше полугода,"6 месяцев мульти, 90 дней пребывания, как и пр..."
167,169,NaN,Москва,ВЦ,Полугодовые,Германия в истории,"Германия под даты (2019), Чехия 1 год (2019-20...",Год и больше,Год
168,170,NaN,Москва,Visametric на Киевской,Годовые и больше,Другие страны,2019 Нидерланды на год,Год и больше,год
169,171,NaN,Москва,ВЦ,Годовые и больше,Германия в истории,За последние 3 года: немецкая виза (11.19-11.2...,Год и больше,2 года


#### Все!
Дальше немного доработал датафрейм вручную и отправился в ноутбук __[germany_awd_visa_analysis](https://github.com/MustDie-green/AWD-forum-visa-parsing/blob/main/germany_awd_visa_analysis.ipynb)__, где анализирую, что же у нас (и у людей с форума) получилось.